# Module 07: Advanced Patterns & Optimization - Practice Notebook

**Level:** Expert  
**Duration:** 4-5 hours  
**Updated:** December 2025 - Production patterns

## Learning Objectives

Expert optimization techniques:
- ✅ Circuit breakers for resilience
- ✅ Caching strategies
- ✅ Performance profiling
- ✅ Cost optimization
- ✅ Advanced error handling


In [ ]:
# Setup
%pip install -q -U langgraph pybreaker redis python-dotenv

from pybreaker import CircuitBreaker
import redis
import time

print('✅ Optimization tools ready!')


---

## Exercise 1: Circuit Breakers 🎯

**Objective:** Prevent cascade failures with circuit breakers.


In [ ]:
# Exercise 1: Circuit breaker pattern

breaker = CircuitBreaker(fail_max=3, timeout_duration=60)

@breaker
def call_external_api():
    """External API call protected by circuit breaker."""
    # Simulate API failure
    import random
    if random.random() < 0.7:  # 70% failure rate
        raise Exception('API Error')
    return {'status': 'success'}

# Test circuit breaker
for i in range(10):
    try:
        result = call_external_api()
        print(f'Request {i+1}: ✅ Success')
    except Exception as e:
        print(f'Request {i+1}: ❌ {e}')
    time.sleep(0.1)

print(f'\nCircuit state: {breaker.current_state}')


---

## Exercise 2: Redis Caching 🎯

**Objective:** Reduce costs with intelligent caching.


In [ ]:
# Exercise 2: LLM response caching

import json
import hashlib

# In production, connect to real Redis
# redis_client = redis.Redis(host='localhost', port=6379, db=0)
# For demo, use dict
cache = {}

def cached_llm_call(prompt: str) -> str:
    # Create cache key
    cache_key = f'llm:{hashlib.md5(prompt.encode()).hexdigest()}'
    
    # Check cache
    if cache_key in cache:
        print(f'✅ Cache HIT for: {prompt[:30]}...')
        return cache[cache_key]
    
    # Simulate LLM call
    print(f' 💰 Cache MISS - calling LLM for: {prompt[:30]}...')
    time.sleep(0.5)  # Simulate latency
    result = f'Response to: {prompt}'
    
    #  Cache result (24hr TTL in production)
    cache[cache_key] = result
    
    return result

# Test caching
prompts = [
    'What is LangGraph?',
    'Explain agents', 
    'What is LangGraph?',  # Duplicate - should hit cache
    'Explain agents'        # Duplicate - should hit cache
]

for prompt in prompts:
    result = cached_llm_call(prompt)

print(f'\n📊 Cache stats: {len(cache)} entries')


---

## Exercise 3: Performance Profiling 🎯

**Objective:** Identify and fix performance bottlenecks.


In [ ]:
# Exercise 3: Profile agent performance

import cProfile
import pstats
from io import StringIO

def slow_function():
    total = 0
    for i in range(1000000):
        total += i
    return total

def fast_function():
    return sum(range(1000000))

# Profile both
profiler = cProfile.Profile()
profiler.enable()

slow_result = slow_function()
fast_result = fast_function()

profiler.disable()

# Print stats
s = StringIO()
ps = pstats.Stats(profiler, stream=s).sort_stats('cumulative')
ps.print_stats(10)
print(s.getvalue())


---

## Exercise 4: Cost Optimization 🎯

**Objective:** Track and reduce LLM costs.


In [ ]:
# Exercise 4: Token usage tracking

class CostTracker:
    def __init__(self):
        self.total_tokens = 0
        self.total_cost = 0
        # GPT-4 pricing (example)
        self.cost_per_1k_tokens = 0.03
    
    def track(self, prompt_tokens: int, completion_tokens: int):
        tokens = prompt_tokens + completion_tokens
        cost = (tokens / 1000) * self.cost_per_1k_tokens
        
        self.total_tokens += tokens
        self.total_cost += cost
        
        return cost
    
    def report(self):
        print(f'📊 Cost Report:')
        print(f'  Total tokens: {self.total_tokens:,}')
        print(f'  Total cost: ${self.total_cost:.4f}')
        print(f'  Avg cost/request: ${self.total_cost/max(1, self.requests):.4f}')

tracker = CostTracker()

# Simulate requests
requests = [
    {'prompt': 500, 'completion': 200},
    {'prompt': 300, 'completion': 150},
    {'prompt': 1000, 'completion': 500}
]

for req in requests:
    cost = tracker.track(req['prompt'], req['completion'])
    print(f'Request cost: ${cost:.4f}')

tracker.report()


---

## Exercise 5: Advanced Error Handling 🎯

**Objective:** Implement graceful degradation.


In [ ]:
# Exercise 5: Retry with exponential backoff

import random

def exponential_backoff_retry(func, max_retries=3):
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            
            wait_time = (2 ** attempt) + random.uniform(0, 1)
            print(f'Attempt {attempt+1} failed: {e}. Retrying in {wait_time:.2f}s...')
            time.sleep(wait_time)

def unreliable_function():
    if random.random() < 0.7:
        raise Exception('Temporary error')
    return 'Success!'

# Test retry logic
try:
    result = exponential_backoff_retry(unreliable_function)
    print(f'✅ {result}')
except Exception as e:
    print(f'❌ Final failure: {e}')


---

## 📚 Summary

Advanced optimization skills:
- ✅ Circuit breakers
- ✅ Intelligent caching
- ✅ Performance profiling
- ✅ Cost tracking
- ✅ Exponential backoff

**Next:** Module 08 - Multi-Agent Systems! 🚀
